# Proyecto 2 Python
## En este proyecto analizaremos un data set de acciones del mercado de valores que estan dentro del S&P 500

In [72]:
import pandas as pd
import numpy as np
import boto3
import psycopg2
import configparser


In [103]:
config = configparser.ConfigParser()
result = config.read('escec.cfg')

In [3]:
s3 = boto3.resource(
    service_name = 's3',
    region_name = 'us-east-1',
    aws_access_key_id = config.get('IAM', 'ACCESS_KEY'),
    aws_secret_access_key = config.get('IAM', 'SECRET_ACCESS_KEY')
)

In [4]:
for bucket in s3.buckets.all():
    S3_BUCKET_NAME = bucket.name
    print(bucket.name)
    
S3_BUCKET_NAME = 'bucketnoemaestria'

bucketnoemaestria


In [5]:
#extraemos todo lo que está en el bucket
remoteFileList = []
for objt in s3.Bucket(S3_BUCKET_NAME).objects.all():
    remoteFileList.append(objt.key)

remoteFileList

['conversion_divisas.xlsx',
 'sp500_companies.csv',
 'sp500_index.csv',
 'sp500_stocks.csv']

In [6]:
import io
for remoteFile in remoteFileList:
    try:
        file = s3.Bucket(S3_BUCKET_NAME).Object(remoteFile).get()
        data = file['Body'].read()
        # Obtener el nombre del archivo CSV sin la extensión
        filename = remoteFile.split('.')[0]
        df = pd.read_csv(io.BytesIO(data), delimiter=',')
        # Renombrar el DataFrame utilizando el nombre del archivo CSV
        globals()[filename] = df
        print("El nombre del nuevo archivo DataFrame es:", filename)
    except Exception as ex:
        print("No es un archivo.")
        print(ex)

No es un archivo.
'utf-8' codec can't decode byte 0x9e in position 14: invalid start byte
El nombre del nuevo archivo DataFrame es: sp500_companies
El nombre del nuevo archivo DataFrame es: sp500_index
El nombre del nuevo archivo DataFrame es: sp500_stocks


In [8]:
print(len(sp500_stocks))

1650343


In [25]:
sp500_stocks.head()

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume
0,2010-01-04 00:00:00-05:00,MMM,59.318886,83.019997,83.449997,82.669998,83.089996,3043700.0
1,2010-01-05 00:00:00-05:00,MMM,58.947342,82.500000,83.230003,81.699997,82.800003,2847000.0
2,2010-01-06 00:00:00-05:00,MMM,59.783295,83.669998,84.599998,83.510002,83.879997,5268500.0
3,2010-01-07 00:00:00-05:00,MMM,59.826176,83.730003,83.760002,82.120003,83.320000,4470100.0
4,2010-01-08 00:00:00-05:00,MMM,60.247749,84.320000,84.320000,83.300003,83.690002,3405800.0


In [31]:
# Convertir la columna de fecha a un objeto datetime de Pandas y establecer la zona horaria en nulo
sp500_stocks["Date"] = pd.to_datetime(sp500_stocks["Date"], utc=True)

In [36]:
sp500_stocks['year'] = pd.DatetimeIndex(sp500_stocks['Date']).year
sp500_stocks['month'] = pd.DatetimeIndex(sp500_stocks['Date']).month
sp500_stocks['quarter'] = pd.DatetimeIndex(sp500_stocks['Date']).quarter
sp500_stocks['day'] = pd.DatetimeIndex(sp500_stocks['Date']).day
sp500_stocks['week'] = pd.DatetimeIndex(sp500_stocks['Date']).week
sp500_stocks['dayofweek'] = pd.DatetimeIndex(sp500_stocks['Date']).dayofweek

c:\Users\noem9\anaconda3\envs\MaestriaDS\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  """


In [37]:
sp500_stocks.head()

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume,year,month,quarter,day,week,dayofweek
0,2010-01-04 05:00:00+00:00,MMM,59.318886,83.019997,83.449997,82.669998,83.089996,3043700.0,2010,1,1,4,1,0
1,2010-01-05 05:00:00+00:00,MMM,58.947342,82.500000,83.230003,81.699997,82.800003,2847000.0,2010,1,1,5,1,1
2,2010-01-06 05:00:00+00:00,MMM,59.783295,83.669998,84.599998,83.510002,83.879997,5268500.0,2010,1,1,6,1,2
3,2010-01-07 05:00:00+00:00,MMM,59.826176,83.730003,83.760002,82.120003,83.320000,4470100.0,2010,1,1,7,1,3
4,2010-01-08 05:00:00+00:00,MMM,60.247749,84.320000,84.320000,83.300003,83.690002,3405800.0,2010,1,1,8,1,4


In [144]:
sp500_stocks2 = sp500_stocks[sp500_stocks['year'] > 2020]
sp500_stocks2 = sp500_stocks2.drop("Date", axis=1)
sp500_stocks2.head()

,Symbol,Adj Close,Close,High,Low,Open,Volume,year,month,quarter,day,week,dayofweek
2769,MMM,164.987518,171.869995,176.199997,170.550003,175.000000,2996200.0,2021,1,1,4,1,0
2770,MMM,164.709137,171.580002,173.250000,170.649994,172.009995,2295300.0,2021,1,1,5,1,1
2771,MMM,167.214630,174.190002,175.570007,172.039993,172.720001,3346400.0,2021,1,1,6,1,2
2772,MMM,162.923615,169.720001,173.460007,166.160004,171.559998,5863400.0,2021,1,1,7,1,3
2773,MMM,159.947754,166.619995,169.539993,164.610001,169.169998,4808100.0,2021,1,1,8,1,4


In [45]:
len(sp500_stocks2)

257536

In [52]:
print(sp500_stocks2.describe())

           Adj Close          Close           High            Low  \
count  256690.000000  256690.000000  256690.000000  256690.000000   
mean      171.139778     171.639418     173.826116     169.396905   
std       282.824523     282.837414     286.509866     279.010962   
min         5.090000       5.090000       5.270000       4.990000   
25%        59.169998      59.639999      60.421248      58.860001   
50%       108.243496     108.690002     110.070000     107.155003   
75%       202.567116     203.410004     205.800003     200.827503   
max      5959.330078    5959.330078    5982.450195    5884.060059   

                Open        Volume           year          month  \
count  256690.000000  2.566900e+05  257536.000000  257536.000000   
mean      171.647329  5.001919e+06    2021.525391       6.468750   
std       282.781511  1.110096e+07       0.533397       3.475226   
min         5.120000  4.300000e+03    2021.000000       1.000000   
25%        59.639999  9.279000e+05    

In [53]:
print(sp500_companies.describe())

       Currentprice     Marketcap        Ebitda  Revenuegrowth  \
count    493.000000  4.930000e+02  4.490000e+02     478.000000   
mean     171.943955  7.212061e+10  6.083028e+09       0.821619   
std      301.251897  1.674742e+11  1.216485e+10      12.528483   
min        5.900000  5.697755e+09 -2.681000e+09      -0.643000   
25%       62.130000  1.692663e+10  1.418000e+09       0.020250   
50%      107.630000  3.119184e+10  2.635000e+09       0.088500   
75%      195.340000  6.450015e+10  5.649200e+09       0.195000   
max     4992.240000  2.143775e+12  1.305410e+11     270.791000   

       Fulltimeemployees      Weight  
count       4.860000e+02  493.000000  
mean        5.605573e+04    0.002028  
std         1.455323e+05    0.004710  
min         1.050000e+02    0.000160  
25%         9.438500e+03    0.000476  
50%         1.926750e+04    0.000877  
75%         5.506750e+04    0.001814  
max         2.300000e+06    0.060294  


In [146]:
dim_date = sp500_stocks2
dim_date = dim_date.drop("Adj Close", axis=1)
dim_date = dim_date.drop("Close", axis=1)
dim_date = dim_date.drop("High", axis=1)
dim_date = dim_date.drop("Low", axis=1)
dim_date = dim_date.drop("Open", axis=1)
dim_date = dim_date.drop("Volume", axis=1)
dim_date = dim_date.drop("Symbol", axis=1)
dim_date['id_fecha'] = dim_date['year'].astype(str) + '_' + dim_date['month'].astype(str) + '_' + dim_date['day'].astype(str)
dim_date.drop_duplicates(inplace=True)
dim_date.head()

,year,month,quarter,day,week,dayofweek,id_fecha
2769,2021,1,1,4,1,0,2021_1_4
2770,2021,1,1,5,1,1,2021_1_5
2771,2021,1,1,6,1,2,2021_1_6
2772,2021,1,1,7,1,3,2021_1_7
2773,2021,1,1,8,1,4,2021_1_8


In [171]:
sp500_stocks2['id'] = sp500_stocks2['year'].astype(str) + '_' + sp500_stocks2['month'].astype(str) + '_' + sp500_stocks2['day'].astype(str) + '_' + sp500_stocks2['Symbol'].astype(str)
sp500_stocks2['id_fecha'] = sp500_stocks2['year'].astype(str) + '_' + sp500_stocks2['month'].astype(str) + '_' + sp500_stocks2['day'].astype(str)

sp500_stocks2.head()

,Symbol,Adj Close,Close,High,Low,Open,Volume,year,month,quarter,day,week,dayofweek,id,id_fecha
2769,MMM,164.987518,171.869995,176.199997,170.550003,175.000000,2996200.0,2021,1,1,4,1,0,2021_1_4_MMM,2021_1_4
2770,MMM,164.709137,171.580002,173.250000,170.649994,172.009995,2295300.0,2021,1,1,5,1,1,2021_1_5_MMM,2021_1_5
2771,MMM,167.214630,174.190002,175.570007,172.039993,172.720001,3346400.0,2021,1,1,6,1,2,2021_1_6_MMM,2021_1_6
2772,MMM,162.923615,169.720001,173.460007,166.160004,171.559998,5863400.0,2021,1,1,7,1,3,2021_1_7_MMM,2021_1_7
2773,MMM,159.947754,166.619995,169.539993,164.610001,169.169998,4808100.0,2021,1,1,8,1,4,2021_1_8_MMM,2021_1_8


In [172]:
sp500_stocks2 = sp500_stocks2.drop("year", axis=1)
sp500_stocks2 = sp500_stocks2.drop("month", axis=1)
sp500_stocks2 = sp500_stocks2.drop("quarter", axis=1)
sp500_stocks2 = sp500_stocks2.drop("day", axis=1)
sp500_stocks2 = sp500_stocks2.drop("week", axis=1)
sp500_stocks2 = sp500_stocks2.drop("dayofweek", axis=1)
sp500_stocks2.head()

,Symbol,Adj Close,Close,High,Low,Open,Volume,id,id_fecha
2769,MMM,164.987518,171.869995,176.199997,170.550003,175.000000,2996200.0,2021_1_4_MMM,2021_1_4
2770,MMM,164.709137,171.580002,173.250000,170.649994,172.009995,2295300.0,2021_1_5_MMM,2021_1_5
2771,MMM,167.214630,174.190002,175.570007,172.039993,172.720001,3346400.0,2021_1_6_MMM,2021_1_6
2772,MMM,162.923615,169.720001,173.460007,166.160004,171.559998,5863400.0,2021_1_7_MMM,2021_1_7
2773,MMM,159.947754,166.619995,169.539993,164.610001,169.169998,4808100.0,2021_1_8_MMM,2021_1_8


In [173]:
sp500_stocks2 = sp500_stocks2.rename(columns={'Symbol':'symbol','Adj Close':'adj_close','Close':'close', 'High':'high','Low':'low','Open':'open','Volume':'volume'})
sp500_stocks2.head()


,symbol,adj_close,close,high,low,open,volume,id,id_fecha
2769,MMM,164.987518,171.869995,176.199997,170.550003,175.000000,2996200.0,2021_1_4_MMM,2021_1_4
2770,MMM,164.709137,171.580002,173.250000,170.649994,172.009995,2295300.0,2021_1_5_MMM,2021_1_5
2771,MMM,167.214630,174.190002,175.570007,172.039993,172.720001,3346400.0,2021_1_6_MMM,2021_1_6
2772,MMM,162.923615,169.720001,173.460007,166.160004,171.559998,5863400.0,2021_1_7_MMM,2021_1_7
2773,MMM,159.947754,166.619995,169.539993,164.610001,169.169998,4808100.0,2021_1_8_MMM,2021_1_8


In [161]:
sp500_index.head()

,sp500,id_fecha,day
2007,3700.65,2021_1_4,4
2008,3726.86,2021_1_5,5
2009,3748.14,2021_1_6,6
2010,3803.79,2021_1_7,7
2011,3824.68,2021_1_8,8


In [162]:
#sp500_index['year'] = pd.DatetimeIndex(sp500_index['id_fecha']).year
#sp500_index['month'] = pd.DatetimeIndex(sp500_index['id_fecha']).month
#sp500_index['day'] = pd.DatetimeIndex(sp500_index['id_fecha']).day
#sp500_index['id_fecha'] = sp500_index['year'].astype(str) + '_' + sp500_index['month'].astype(str) + '_' + sp500_index['day'].astype(str)
#sp500_index = sp500_index.drop("year", axis=1)
#sp500_index = sp500_index.drop("month", axis=1)
#sp500_index = sp500_index.drop("Date", axis=1)
sp500_index = sp500_index.drop("day", axis=1)
sp500_index.head()

,sp500,id_fecha
2007,3700.65,2021_1_4
2008,3726.86,2021_1_5
2009,3748.14,2021_1_6
2010,3803.79,2021_1_7
2011,3824.68,2021_1_8


In [155]:
sp500_index = sp500_index.rename(columns={'S&P500':'sp500'})
# Convertir la columna de fecha a formato datetime
sp500_index['id_fecha'] = pd.to_datetime(sp500_index['id_fecha'], format='%Y_%m_%d')
sp500_index = sp500_index[sp500_index['id_fecha'].dt.year > 2020]
sp500_index.head()

,sp500,id_fecha
2007,3700.65,2021-01-04
2008,3726.86,2021-01-05
2009,3748.14,2021-01-06
2010,3803.79,2021-01-07
2011,3824.68,2021-01-08


In [164]:
sp500_index.drop_duplicates(inplace=True)

In [92]:
print(sp500_stocks2.describe())

           Adj Close          Close           High            Low  \
count  256690.000000  256690.000000  256690.000000  256690.000000   
mean      171.139778     171.639418     173.826116     169.396905   
std       282.824523     282.837414     286.509866     279.010962   
min         5.090000       5.090000       5.270000       4.990000   
25%        59.169998      59.639999      60.421248      58.860001   
50%       108.243496     108.690002     110.070000     107.155003   
75%       202.567116     203.410004     205.800003     200.827503   
max      5959.330078    5959.330078    5982.450195    5884.060059   

                Open        Volume  
count  256690.000000  2.566900e+05  
mean      171.647329  5.001919e+06  
std       282.781511  1.110096e+07  
min         5.120000  4.300000e+03  
25%        59.639999  9.279000e+05  
50%       108.629997  1.922600e+06  
75%       203.410004  4.439700e+06  
max      5977.609863  3.536488e+08  


In [93]:
print(sp500_companies.describe())

       Currentprice     Marketcap        Ebitda  Revenuegrowth  \
count    493.000000  4.930000e+02  4.490000e+02     478.000000   
mean     171.943955  7.212061e+10  6.083028e+09       0.821619   
std      301.251897  1.674742e+11  1.216485e+10      12.528483   
min        5.900000  5.697755e+09 -2.681000e+09      -0.643000   
25%       62.130000  1.692663e+10  1.418000e+09       0.020250   
50%      107.630000  3.119184e+10  2.635000e+09       0.088500   
75%      195.340000  6.450015e+10  5.649200e+09       0.195000   
max     4992.240000  2.143775e+12  1.305410e+11     270.791000   

       Fulltimeemployees      Weight  
count       4.860000e+02  493.000000  
mean        5.605573e+04    0.002028  
std         1.455323e+05    0.004710  
min         1.050000e+02    0.000160  
25%         9.438500e+03    0.000476  
50%         1.926750e+04    0.000877  
75%         5.506750e+04    0.001814  
max         2.300000e+06    0.060294  


In [94]:
print(sp500_index.describe())

            S&P500
count  2519.000000
mean   2750.910854
std     872.383979
min    1470.680000
25%    2052.630000
50%    2596.640000
75%    3296.530000
max    4796.560000


In [134]:
sp500_companies = sp500_companies.rename(columns={'Exchange':'exchange','Symbol':'symbol','Shortname':'shortname','Longname':'longname', 'Sector':'sector','Industry':'industry','Currentprice':'currentprice',	'Marketcap':'marketcap','Ebitda':'ebitda','Revenuegrowth':'revenuegrowth','City':'city','State':'state','Country':'country', 'Fulltimeemployees':'fulltimeemployees','Longbusinesssummary':'longbusinesssummary','Weight':'weight'})
sp500_companies = sp500_companies.drop("longbusinesssummary", axis=1)

In [135]:
sp500_companies.head()

,exchange,symbol,shortname,longname,sector,industry,currentprice,marketcap,ebitda,revenuegrowth,city,state,country,fulltimeemployees,weight
0,NMS,AAPL,Apple Inc.,Apple Inc.,Technology,Consumer Electronics,134.76,2143775490048,1.305410e+11,0.081,Cupertino,CA,United States,164000.0,0.060294
1,NMS,MSFT,Microsoft Corporation,Microsoft Corporation,Technology,Software—Infrastructure,239.23,1796134010880,9.798300e+10,0.124,Redmond,WA,United States,221000.0,0.050516
2,NMS,GOOGL,Alphabet Inc.,Alphabet Inc.,Communication Services,Internet Content & Information,92.12,1197108625408,9.373300e+10,0.061,Mountain View,CA,United States,186779.0,0.033669
3,NMS,GOOG,Alphabet Inc.,Alphabet Inc.,Communication Services,Internet Content & Information,92.80,1195709431808,9.373300e+10,0.061,Mountain View,CA,United States,186779.0,0.033629
4,NMS,AMZN,"Amazon.com, Inc.","Amazon.com, Inc.",Consumer Cyclical,Internet Retail,98.12,1000990834688,5.154900e+10,0.147,Seattle,WA,United States,1544000.0,0.028153


In [181]:
valid_symbols = list(sp500_companies['symbol'])
sp500_stocks2 = sp500_stocks2[sp500_stocks2['symbol'].isin(valid_symbols)]

In [104]:
rdsIdentifier = 'stocks' #nombre de la instancia
aws_conn = boto3.client('rds', aws_access_key_id=config.get('IAM', 'ACCESS_KEY'),
                    aws_secret_access_key=config.get('IAM', 'SECRET_ACCESS_KEY'),
                    region_name='us-east-1')


In [110]:
rdsInstanceIds = []

response = aws_conn.describe_db_instances()
for resp in response['DBInstances']:
    rdsInstanceIds.append(resp['DBInstanceIdentifier'])
    db_instance_status = resp['DBInstanceStatus']

print(f"DBInstanceIds {rdsInstanceIds}")

DBInstanceIds ['banco-db', 'banco-db-v1', 'banco-db-v2', 'dw-db']


In [111]:
try:
    response = aws_conn.create_db_instance(
            AllocatedStorage=10,
            DBName=config.get('RDS', 'DB_NAME'),
            DBInstanceIdentifier=rdsIdentifier,
            DBInstanceClass="db.t3.micro",
            Engine="postgres",
            MasterUsername=config.get('RDS', 'DB_USER'),
            MasterUserPassword=config.get('RDS', 'DB_PASSWORD'),
            Port=int(config.get('RDS', 'DB_PORT')),
            VpcSecurityGroupIds=[config.get('VPC', 'SECURITY_GROUP')],
            PubliclyAccessible=True
        )
    print(response)
except aws_conn.exceptions.DBInstanceAlreadyExistsFault as ex:
    print("La Instancia de Base de Datos ya Existe.")

{'DBInstance': {'DBInstanceIdentifier': 'stocks', 'DBInstanceClass': 'db.t3.micro', 'Engine': 'postgres', 'DBInstanceStatus': 'creating', 'MasterUsername': 'postgres', 'DBName': 'stocks', 'AllocatedStorage': 10, 'PreferredBackupWindow': '09:42-10:12', 'BackupRetentionPeriod': 1, 'DBSecurityGroups': [], 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-0696ee2cd262294a6', 'Status': 'active'}], 'DBParameterGroups': [{'DBParameterGroupName': 'default.postgres14', 'ParameterApplyStatus': 'in-sync'}], 'DBSubnetGroup': {'DBSubnetGroupName': 'default', 'DBSubnetGroupDescription': 'default', 'VpcId': 'vpc-0e80e4c0b6ac301bb', 'SubnetGroupStatus': 'Complete', 'Subnets': [{'SubnetIdentifier': 'subnet-08e06dd497c9855c1', 'SubnetAvailabilityZone': {'Name': 'us-east-1d'}, 'SubnetOutpost': {}, 'SubnetStatus': 'Active'}, {'SubnetIdentifier': 'subnet-07144da968183968e', 'SubnetAvailabilityZone': {'Name': 'us-east-1c'}, 'SubnetOutpost': {}, 'SubnetStatus': 'Active'}, {'SubnetIdentifier': 'subnet-05500a7f

In [101]:
try:
     instances = aws_conn.describe_db_instances(DBInstanceIdentifier=rdsIdentifier)
     RDS_HOST = instances.get('DBInstances')[0].get('Endpoint').get('Address')
     print(RDS_HOST)
except Exception as ex:
     print("La instancia de base de datos no existe o aun no se ha terminado de crear.")
     print(ex)

stocks.csg44mdgnhhv.us-east-1.rds.amazonaws.com


In [112]:
import sql_queries
try:
    db_conn = psycopg2.connect(
        database=config.get('RDS', 'DB_NAME'), 
        user=config.get('RDS', 'DB_USER'),
        password=config.get('RDS', 'DB_PASSWORD'), 
        host=RDS_HOST,
        port=config.get('RDS', 'DB_PORT')
    )

    cursor = db_conn.cursor()
    cursor.execute(sql_queries.DDL_QUERY)
    db_conn.commit()
    print("Base de Datos Creada Exitosamente")
except Exception as ex:
    print("ERROR: Error al crear la base de datos.")
    print(ex)

Base de Datos Creada Exitosamente


In [141]:
postgres_driver = f"""postgresql://{config.get('RDS', 'DB_USER')}:{config.get('RDS', 'DB_PASSWORD')}@{RDS_HOST}:{config.get('RDS', 'DB_PORT')}/{config.get('RDS', 'DB_NAME')}"""    
try:
          response = sp500_companies.to_sql('dim_companies', postgres_driver, index=False, if_exists='append')
          print(f'Se han insertado {response} nuevos registros.' )
except Exception as ex:
          print(ex)

Se han insertado None nuevos registros.


In [165]:
postgres_driver = f"""postgresql://{config.get('RDS', 'DB_USER')}:{config.get('RDS', 'DB_PASSWORD')}@{RDS_HOST}:{config.get('RDS', 'DB_PORT')}/{config.get('RDS', 'DB_NAME')}"""    
try:
          response = dim_date.to_sql('dim_date', postgres_driver, index=False, if_exists='append')
          print(f'Se han insertado {response} nuevos registros.' )
except Exception as ex:
          print(ex)

(psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "dim_date_pkey"
DETAIL:  Key (id_fecha)=(2021_1_4) already exists.

[SQL: INSERT INTO dim_date (year, month, quarter, day, week, dayofweek, id_fecha) VALUES (%(year)s, %(month)s, %(quarter)s, %(day)s, %(week)s, %(dayofweek)s, %(id_fecha)s)]
[parameters: ({'year': 2021, 'month': 1, 'quarter': 1, 'day': 4, 'week': 1, 'dayofweek': 0, 'id_fecha': '2021_1_4'}, {'year': 2021, 'month': 1, 'quarter': 1, 'day': 5, 'week': 1, 'dayofweek': 1, 'id_fecha': '2021_1_5'}, {'year': 2021, 'month': 1, 'quarter': 1, 'day': 6, 'week': 1, 'dayofweek': 2, 'id_fecha': '2021_1_6'}, {'year': 2021, 'month': 1, 'quarter': 1, 'day': 7, 'week': 1, 'dayofweek': 3, 'id_fecha': '2021_1_7'}, {'year': 2021, 'month': 1, 'quarter': 1, 'day': 8, 'week': 1, 'dayofweek': 4, 'id_fecha': '2021_1_8'}, {'year': 2021, 'month': 1, 'quarter': 1, 'day': 11, 'week': 2, 'dayofweek': 0, 'id_fecha': '2021_1_11'}, {'year': 2021, 'month': 1, 'quarter': 1, 'da

In [166]:
postgres_driver = f"""postgresql://{config.get('RDS', 'DB_USER')}:{config.get('RDS', 'DB_PASSWORD')}@{RDS_HOST}:{config.get('RDS', 'DB_PORT')}/{config.get('RDS', 'DB_NAME')}"""    
try:
          response = sp500_index.to_sql('sp500', postgres_driver, index=False, if_exists='append')
          print(f'Se han insertado {response} nuevos registros.' )
except Exception as ex:
          print(ex)

Se han insertado None nuevos registros.


In [182]:
postgres_driver = f"""postgresql://{config.get('RDS', 'DB_USER')}:{config.get('RDS', 'DB_PASSWORD')}@{RDS_HOST}:{config.get('RDS', 'DB_PORT')}/{config.get('RDS', 'DB_NAME')}"""    
try:
          response = sp500_stocks2.to_sql('stock_daily', postgres_driver, index=False, if_exists='append')
          print(f'Se han insertado {response} nuevos registros.' )
except Exception as ex:
          print(ex)

Se han insertado None nuevos registros.
